In [2]:
import sys
sys.path.append('/home/student/ROI/SparkProgram')
from initspark import *
sc, spark, conf = initspark()

import pandas as pd
import matplotlib as mp
import numpy
from matplotlib import pyplot as plt
from IPython.display import display


In [12]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import DecisionTreeClassifier
#lrModel = LogisticRegression.load('bank_csv_LR.trainedModel')
#gbtModel = GBTClassifier.load('bank_csv_GBT.trainedModel')
dtModel = DecisionTreeClassifier.load('bank_csv_DT_trainedModel')



Py4JJavaError: An error occurred while calling o115.load.
: java.lang.NoSuchMethodException: org.apache.spark.ml.classification.DecisionTreeClassificationModel.<init>(java.lang.String)
	at java.lang.Class.getConstructor0(Class.java:3082)
	at java.lang.Class.getConstructor(Class.java:1825)
	at org.apache.spark.ml.util.DefaultParamsReader.load(ReadWrite.scala:496)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(Gatew

In [8]:
catindex = {'job': ['management', 'blue-collar', 'technician', 'admin.', 'services', 'retired', 'self-employed', 'student', 'unemployed', 'entrepreneur', 'housemaid', 'unknown']
              , 'marital': ['married', 'single', 'divorced']
              , 'education': ['secondary', 'tertiary', 'primary', 'unknown']
              , 'default': ['no', 'yes']
              , 'housing': ['no', 'yes']
              , 'loan': ['no', 'yes']
              , 'contact': ['cellular', 'unknown', 'telephone']
              , 'poutcome': ['unknown', 'failure', 'success', 'other']}

def convertBankData(age, job, marital, education, default, balance, housing, loan, contact, duration, campaign, pdays, previous, poutcome):
    return (age
            , catindex['job'].index(job)
            , catindex['marital'].index(marital)
            , catindex['education'].index(education)
            , catindex['default'].index(default)
            , balance
            , catindex['housing'].index(housing)
            , catindex['loan'].index(loan)
            , catindex['contact'].index(contact)
            , duration, campaign, pdays, previous
            , catindex['poutcome'].index(poutcome))


x = convertBankData(age=59, job='admin.', marital='married', education='secondary', default='no', 
     balance=2343, housing='yes', loan='no', contact='unknown', duration=1042, 
                    campaign=1, pdays=-1, previous=0, poutcome='unknown')
print(x)


(59, 3, 0, 0, 0, 2343, 1, 0, 1, 1042, 1, -1, 0, 0)


In [17]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

df = spark.createDataFrame([
(0, "a"),
(1, "b"),
(2, "c"),
(3, "a"),
(4, "a"),
(5, "c"),
(6, "d"),
(7, "e"),
(8, "f")
], ["id", "category"])

stringIndexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
model = stringIndexer.fit(df)
indexed = model.transform(df)

encoder = OneHotEncoder(inputCol="categoryIndex", outputCol="categoryVec")
encoded = encoder.transform(indexed)
encoded.show()

#0  -> 10
#1  -> 01
#2  -> 00
#3  

+---+--------+-------------+-------------+
| id|category|categoryIndex|  categoryVec|
+---+--------+-------------+-------------+
|  0|       a|          0.0|(5,[0],[1.0])|
|  1|       b|          4.0|(5,[4],[1.0])|
|  2|       c|          1.0|(5,[1],[1.0])|
|  3|       a|          0.0|(5,[0],[1.0])|
|  4|       a|          0.0|(5,[0],[1.0])|
|  5|       c|          1.0|(5,[1],[1.0])|
|  6|       d|          5.0|    (5,[],[])|
|  7|       e|          2.0|(5,[2],[1.0])|
|  8|       f|          3.0|(5,[3],[1.0])|
+---+--------+-------------+-------------+

